In [11]:
import torch 
import torch.optim as optim

import seq2seq_multilayer_gru
from sequence_model_trainer import TrainModel

from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset
from torchtext.data import Iterator, BucketIterator

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
MASKED_TEXT = Field(
                sequential=True,
                tokenize=lambda x: x.split(), 
             )

TARGET_TEXT = Field(
                sequential=True,
                tokenize=lambda x: x.split(), 
             )

# LABEL = LabelField(dtype=torch.long)

fields = [('id', None), ('keywords', MASKED_TEXT), ('target', TARGET_TEXT)]

In [3]:
# %cd /content/gdrive/My\ Drive/Colab\ Notebooks/Datasets
train, val, test = TabularDataset.splits(
                            path='C:\\Users\\under\\Datasets\\Conversations\\Microsoft Challenge\\Masked Corpus',
                            train='train.csv',
                            validation='val.csv',
                            test='test.csv',
                            format='csv',
                            skip_header=True,
                            fields=fields
                    )

In [4]:
MASKED_TEXT.build_vocab(train)
TARGET_TEXT.build_vocab(train)
# LABEL.build_vocab(train)

In [5]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iter, val_iter, test_iter = BucketIterator.splits(
                            (train, val, test),
                            batch_size=BATCH_SIZE,
                            device=device,
                            sort_key=lambda x: len(x.keywords))

In [12]:
# EMB_DIM=bert.config.to_dict()['hidden_size']
EMB_DIM=256
ENC_INPUT_DIM=len(MASKED_TEXT.vocab)
DEC_INPUT_DIM=len(TARGET_TEXT.vocab)
OUTPUT_DIM=DEC_INPUT_DIM
N_LAYER=4
HID_DIM=1024
DROPOUT=0.2

model = seq2seq_multilayer_gru.Seq2Seq(
              enc_input_dim=ENC_INPUT_DIM,
              dec_input_dim=DEC_INPUT_DIM,
              emb_dim=EMB_DIM,
              enc_hid_dim=HID_DIM,
              dec_hid_dim=HID_DIM,
              n_layers=N_LAYER,
              output_dim=OUTPUT_DIM,    
              device=device,
              dropout=DROPOUT
         ).to(device)

In [13]:
LEARNING_RATE = 0.0001

adam = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
cross_e = torch.nn.CrossEntropyLoss()

In [14]:
trainer = TrainModel(
            model=model,
            train_iterator=train_iter,
            val_iterator=val_iter,
            optimizer=adam,
            criterion=cross_e,
            output_dim=OUTPUT_DIM
          )

In [15]:
N_EPOCHS = 200
CLIP = 1
trainer.epoch(n_epochs=N_EPOCHS, clip=CLIP, model_name='seq2seq-gru.pt')

Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hidden: torch.Size([4, 128, 1024])
Seq2Seq hid

KeyboardInterrupt: 